In [16]:
import os
from dotenv import load_dotenv
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI
from langchain.vectorstores import FAISS
from langchain.document_loaders import GutenbergLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.evaluation.qa import QAEvalChain

In [17]:
load_dotenv()

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY_4")
azure_openai_api_endpoint = os.getenv("AZURE_OPENAI_API_ENDPOINT_4")
azure_deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME_4")

# Vérification que la clé API LangSmith est définie
if not os.getenv("LANGCHAIN_API_KEY"):
    raise ValueError("La clé API LangSmith n'est pas définie")

langchain_project = os.getenv("LANGCHAIN_PROJECT")

In [18]:
embedding_model = AzureOpenAIEmbeddings(
    openai_api_key=azure_openai_api_key,
    azure_endpoint=azure_openai_api_endpoint,
    deployment="text-embedding-3-large",
    openai_api_version="2023-05-15",
    chunk_size=500
)

llm = AzureChatOpenAI(
    api_key=azure_openai_api_key,
    api_version="2023-05-15",
    azure_endpoint=azure_openai_api_endpoint,
    deployment_name=azure_deployment_name,
    temperature=0
)

In [19]:
from langchain.vectorstores import FAISS

vectorstore_path = "book_index"  # On reprend le vectorstore sauvegardé dans le premier notebook

if os.path.exists(vectorstore_path):
    vectorstore = FAISS.load_local(
        vectorstore_path,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True  
    )
    print(f"Vectorstore chargé depuis {vectorstore_path}.")
else:
    print(f"Le vectorstore n'a pas été trouvé à l'emplacement {vectorstore_path}.")

Vectorstore chargé depuis book_index.


In [20]:
retriever = vectorstore.as_retriever()

In [21]:
PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    Vous êtes un assistant littéraire expert.

    Utilisez les informations suivantes pour répondre à la question de manière précise et concise.

    Informations :
    {context}

    Question :
    {question}

    Réponse :
    """
)

In [22]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": PROMPT}
)

In [23]:
def evaluate_with_langsmith(qa_chain, examples):
    eval_chain = QAEvalChain.from_llm(llm)
    predictions = []

    for example in examples:
        answer = qa_chain.run(example["question"])
        predictions.append({"answer": answer})

    graded_outputs = eval_chain.evaluate(
        examples=examples,
        predictions=predictions,
        question_key="question",
        answer_key="answer",
        prediction_key="answer"
    )
    return graded_outputs, predictions

In [24]:
examples = [
    {
        "question": "Quels sont les thèmes principaux du livre 'Les Misérables' ?",
        "answer": "Les thèmes principaux incluent la justice sociale, la rédemption, l'amour, et la pauvreté."
    },
    {
        "question": "Qui sont les personnages principaux dans le résumé du livre 'Le Roi des Étudiants' ?",
        "answer": "Les personnages principaux sont Paul Champfort et ses camarades de lycée"
    },
    {
        "question": "Quels sont les thèmes principaux du livre 'Les fleurs du mal' ?",
        "answer": "Les thèmes principaux sont la mélancolie, la beauté, la société, et la lutte morale."
    },
    {
        "question": "Résume moi 'Fairy Tales from the Arabian Nights' ?",
        "answer": "C'est une collection de contes fantastiques inspirés du folklore moyen-oriental dont Les histoires explorent des thèmes de l'amour, de la magie et de l'aventure"
    }
]

In [25]:
# Exécuter l'évaluation
graded_outputs, predictions = evaluate_with_langsmith(qa_chain, examples)

/home/yaskck/.pyenv/versions/3.10.15/envs/langchain_env/lib/python3.10/site-packages/langchain_community/chat_models/openai.py:173: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")


In [26]:
# Résultats
for idx, result in enumerate(graded_outputs):
    print("=" * 50)
    print(f"Result dict: {result}")
    print(f"Question {idx + 1}: {examples[idx]['question']}")
    print(f"Réponse générée: {predictions[idx]['answer']}")
    print(f"Réponse attendue: {examples[idx]['answer']}")
    print(f"Évaluation: {result['results']}")
    print("=" * 50)

Result dict: {'results': 'CORRECT'}
Question 1: Quels sont les thèmes principaux du livre 'Les Misérables' ?
Réponse générée: Les thèmes principaux de "Les Misérables" sont la justice, la moralité, la rédemption, et la lutte entre le bien et le mal. Le roman explore également les problèmes sociaux, notamment la pauvreté et la marginalisation, à travers les vies de ses personnages, en particulier Jean Valjean. L'œuvre aborde aussi les sacrifices, l'espoir d'un avenir meilleur, et les dilemmes moraux dans un contexte de bouleversements historiques et sociaux.
Réponse attendue: Les thèmes principaux incluent la justice sociale, la rédemption, l'amour, et la pauvreté.
Évaluation: CORRECT
Result dict: {'results': 'INCORRECT'}
Question 2: Qui sont les personnages principaux dans le résumé du livre 'Le Roi des Étudiants' ?
Réponse générée: Les personnages principaux dans le résumé du livre "Le Roi des Étudiants" sont Paul Champfort et un groupe d'étudiants en médecine à Québec.
Réponse attend